https://docs.fenicsproject.org/dolfinx/v0.7.3/python/demos/demo_stokes.html#nested-matrix-solver  
https://github.com/unifem/fenics-notes/blob/master/notebooks/advection-diffusion-reaction.ipynb  
https://fenicsproject.org/pub/tutorial/html/._ftut1010.html  


In [6]:
from mpi4py import MPI
from petsc4py import PETSc

import numpy as np

import ufl
from basix.ufl import element
from dolfinx import fem, io, mesh, plot, la
from dolfinx.fem.petsc import assemble_matrix_block, assemble_vector_block, LinearProblem
from dolfinx.io import XDMFFile
from ufl import div, dx, grad, inner

import pyvista


In [7]:
# Create mesh
msh = mesh.create_rectangle(MPI.COMM_WORLD,
                       [np.array([0, 0]), np.array([1, 1])],
                       [16, 16],
                       mesh.CellType.triangle)


# Function to mark x = 0, x = 1 and y = 0
def noslip_boundary(x):
    return np.logical_or(np.logical_or(np.isclose(x[0], 0.0),
                                       np.isclose(x[0], 1.0)),
                         np.isclose(x[1], 0.0))


# Function to mark the lid (y = 1)
def lid(x):
    return np.isclose(x[1], 1.0)


# Lid velocity
def lid_velocity_expression(x):
    return np.stack((np.ones(x.shape[1]), np.zeros(x.shape[1]))) * 1

In [20]:
P2 = element("Lagrange", msh.basix_cell(), 2, shape=(msh.geometry.dim,))
P1 = element("Lagrange", msh.basix_cell(), 1)
V, Q = fem.functionspace(msh, P2), fem.functionspace(msh, P1)

In [9]:
# define boundary conditions

# No-slip boundary condition for velocity field (`V`) on boundaries
# where x = 0, x = 1, and y = 0
noslip = np.zeros(msh.geometry.dim, dtype=PETSc.ScalarType)
facets = mesh.locate_entities_boundary(msh, 1, noslip_boundary)
bc0 = fem.dirichletbc(noslip, fem.locate_dofs_topological(V, 1, facets), V)

# Driving velocity condition u = (1, 0) on top boundary (y = 1)
lid_velocity = fem.Function(V)
lid_velocity.interpolate(lid_velocity_expression)
facets = mesh.locate_entities_boundary(msh, 1, lid)
bc1 = fem.dirichletbc(lid_velocity, fem.locate_dofs_topological(V, 1, facets))

# Collect Dirichlet boundary conditions
bcs = [bc0, bc1]

In [16]:
# Define variational problem
(u, p) = ufl.TrialFunction(V), ufl.TrialFunction(Q)
(v, q) = ufl.TestFunction(V), ufl.TestFunction(Q)
f = fem.Constant(msh, (PETSc.ScalarType(0), PETSc.ScalarType(0)))

a = fem.form([[inner(grad(u), grad(v)) * dx, inner(p, div(v)) * dx],
          [inner(div(u), q) * dx, None]])
L = fem.form([inner(f, v) * dx, inner(fem.Constant(msh, PETSc.ScalarType(0)), q) * dx])

In [11]:
a_p11 = fem.form(inner(p, q) * dx)
a_p = [[a[0][0], None],
       [None, a_p11]]

In [18]:
LinearProblem(a,L)

Exception ignored in: <function LinearProblem.__del__ at 0x7fcb057c7ac0>
Traceback (most recent call last):
  File "/home/wrk/miniconda3/envs/ProjEnv/lib/python3.10/site-packages/dolfinx/fem/petsc.py", line 627, in __del__
    self._solver.destroy()
AttributeError: 'LinearProblem' object has no attribute '_solver'


TypeError: LinearProblem.__init__() missing 1 required positional argument: 'L'

In [7]:
"""Solve the Stokes problem using nest matrices and an iterative solver."""

# Assemble nested matrix operators
A = fem.petsc.assemble_matrix_nest(a, bcs=bcs)
A.assemble()

# Create a nested matrix P to use as the preconditioner. The
# top-left block of P is shared with the top-left block of A. The
# bottom-right diagonal entry is assembled from the form a_p11:
P11 = fem.petsc.assemble_matrix(a_p11, [])
P = PETSc.Mat().createNest([[A.getNestSubMatrix(0, 0), None], [None, P11]])
P.assemble()

A00 = A.getNestSubMatrix(0, 0)
A00.setOption(PETSc.Mat.Option.SPD, True)

P00, P11 = P.getNestSubMatrix(0, 0), P.getNestSubMatrix(1, 1)
P00.setOption(PETSc.Mat.Option.SPD, True)
P11.setOption(PETSc.Mat.Option.SPD, True)

# Assemble right-hand side vector
b = fem.petsc.assemble_vector_nest(L)

# Modify ('lift') the RHS for Dirichlet boundary conditions
fem.petsc.apply_lifting_nest(b, a, bcs=bcs)

# Sum contributions for vector entries that are share across
# parallel processes
for b_sub in b.getNestSubVecs():
    b_sub.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)

# Set Dirichlet boundary condition values in the RHS vector
bcs0 = fem.bcs_by_block(fem.extract_function_spaces(L), bcs)
fem.petsc.set_bc_nest(b, bcs0)

# The pressure field is determined only up to a constant. We supply
# a vector that spans the nullspace to the solver, and any component
# of the solution in this direction will be eliminated during the
# solution process.
null_vec = fem.petsc.create_vector_nest(L)

# Set velocity part to zero and the pressure part to a non-zero
# constant
null_vecs = null_vec.getNestSubVecs()
null_vecs[0].set(0.0), null_vecs[1].set(1.0)

# Normalize the vector that spans the nullspace, create a nullspace
# object, and attach it to the matrix
null_vec.normalize()
nsp = PETSc.NullSpace().create(vectors=[null_vec])
assert nsp.test(A)
A.setNullSpace(nsp)

# Create a MINRES Krylov solver and a block-diagonal preconditioner
# using PETSc's additive fieldsplit preconditioner
ksp = PETSc.KSP().create(msh.comm)
ksp.setOperators(A, P)
ksp.setType("minres")
ksp.setTolerances(rtol=1e-9)
ksp.getPC().setType("fieldsplit")
ksp.getPC().setFieldSplitType(PETSc.PC.CompositeType.ADDITIVE)

# Define the matrix blocks in the preconditioner with the velocity
# and pressure matrix index sets
nested_IS = P.getNestISs()
ksp.getPC().setFieldSplitIS(("u", nested_IS[0][0]), ("p", nested_IS[0][1]))

# Set the preconditioners for each block. For the top-left
# Laplace-type operator we use algebraic multigrid. For the
# lower-right block we use a Jacobi preconditioner. By default, GAMG
# will infer the correct near-nullspace from the matrix block size.
ksp_u, ksp_p = ksp.getPC().getFieldSplitSubKSP()
ksp_u.setType("preonly")
ksp_u.getPC().setType("gamg")
ksp_p.setType("preonly")
ksp_p.getPC().setType("jacobi")

# Create finite element {py:class}`Function <dolfinx.fem.Function>`s
# for the velocity (on the space `V`) and for the pressure (on the
# space `Q`). The vectors for `u` and `p` are combined to form a
# nested vector and the system is solved.
u, p = fem.Function(V), fem.Function(Q)
x = PETSc.Vec().createNest([la.create_petsc_vector_wrap(u.x),
                            la.create_petsc_vector_wrap(p.x)])
ksp.solve(b, x)

# Save solution to file in XDMF format for visualization, e.g. with
# ParaView. Before writing to file, ghost values are updated using
# `scatter_forward`.
with XDMFFile(MPI.COMM_WORLD, "out_stokes/velocity.xdmf", "w") as ufile_xdmf:
    u.x.scatter_forward()
    P1 = element("Lagrange", msh.basix_cell(), 1, shape=(msh.geometry.dim,))
    u1 = fem.Function(fem.functionspace(msh, P1))
    u1.interpolate(u)
    ufile_xdmf.write_mesh(msh)
    ufile_xdmf.write_function(u1)

with XDMFFile(MPI.COMM_WORLD, "out_stokes/pressure.xdmf", "w") as pfile_xdmf:
    p.x.scatter_forward()
    pfile_xdmf.write_mesh(msh)
    pfile_xdmf.write_function(p)

# Compute norms of the solution vectors
norm_u = u.x.norm()
norm_p = p.x.norm()
if MPI.COMM_WORLD.rank == 0:
    print(f"(A) Norm of velocity coefficient vector (nested, iterative): {norm_u}")
    print(f"(A) Norm of pressure coefficient vector (nested, iterative): {norm_p}")



(A) Norm of velocity coefficient vector (nested, iterative): 9.259397838352873
(A) Norm of pressure coefficient vector (nested, iterative): 0.24049500012348662


In [10]:
pyvista.set_jupyter_backend('client')

pyvista.start_xvfb()
topology, cell_types, geometry = plot.vtk_mesh(V)
values = np.zeros((geometry.shape[0], 3), dtype=np.float64)
values[:, :len(u)] = u.x.array.real.reshape((geometry.shape[0], len(u)))

# Create a point cloud of glyphs
function_grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
function_grid["u"] = values
glyphs = function_grid.glyph(orient="u", factor=0.2)

# Create pyvist grid to show pressure
p_topology, p_cell_types, p_geometry = plot.vtk_mesh(Q)
pressure_grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
pressure_grid.point_data["p"] = values
pressure_grid.set_active_scalars("p")

# Create a pyvista-grid for the mesh
grid = pyvista.UnstructuredGrid(*plot.vtk_mesh(msh, msh.topology.dim))

# Create plotter
plotter = pyvista.Plotter(notebook=True)
#plotter.add_mesh(pressure_grid, show_edges=True)
plotter.add_mesh(glyphs)
plotter.view_xy()

plotter.show()
#fig_as_array = plotter.screenshot("glyphs2.png")

Widget(value='<iframe src="http://localhost:37493/index.html?ui=P_0x7f08b78f46d0_2&reconnect=auto" class="pyvi…

In [9]:
with io.VTXWriter(msh.comm, "solution.bp", [u, p], engine="BP4") as vtx:
  vtx.write(0.0)

RuntimeError: All functions in VTXWriter must have the same element type.

# TODO
- calculate reynolds number
- visualise in different ways
- ADR